In [106]:
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Embedding, CuDNNLSTM, SpatialDropout1D
import tensorflow as tf
from keras import regularizers

In [107]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [108]:
import pickle

emojis = pd.read_csv('data/Emoji_Sentiment_Data_v1.0.csv')
sentiment = pd.read_csv('data/sentiment.csv')
with open('data/sentiment_texts.pickle', 'rb') as f:
    texts = pickle.load(f)

x, y = texts.MessageText, texts.SentimentScore

In [109]:
from sklearn.model_selection import train_test_split

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)
x = pad_sequences(x)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_test).values

In [110]:
# 
# 
# embed_dim = 128
# lstm_out = 32
# 
# model = Sequential()
# model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
# model.add(SpatialDropout1D(0.2))
# model.add(CuDNNLSTM(lstm_out, kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-2)))
# model.add(Dense(6, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.build(X_train.shape[1])
# print(model.summary())

In [111]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [112]:
from matplotlib import pyplot

# batch_size = 32
# print(X_train.shape, y_train.shape)
# history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1, validation_data=(X_test, y_val))
# 
# pyplot.plot(history.history['loss'])
# pyplot.plot(history.history['val_loss'])
# pyplot.title('model train vs validation loss')
# pyplot.ylabel('loss')
# pyplot.xlabel('epoch')
# pyplot.legend(['train', 'validation'], loc='upper right')
# pyplot.show()

In [113]:
from sklearn.metrics import accuracy_score, f1_score


def final_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    return (f1 + accuracy) / 2

# y_pred = model.predict(X_test).argmax(axis=1)
# print(final_score(y_test, y_pred))

In [114]:
from keras.layers import Bidirectional, Conv1D, MaxPooling1D
from keras.optimizers import RMSprop, Adam
import optuna
from optuna.integration import TFKerasPruningCallback


def objective(trial):
    #embed_dim = trial.suggest_int('embed_dim', 100, 110, step=2)
    embed_dim = 105
    #lstm1_out = trial.suggest_int('lstm1_out', 45, 55, step=2)
    lstm1_out = 47
    #lstm2_out = trial.suggest_int('lstm2_out', 20, 100, step=10)

    #spat_dropout = trial.suggest_float('dropout', 0, 0.15)
    spat_dropout = 0.1

    #l1 = trial.suggest_float('l1', 0.003, 0.007)
    l1 = 0.05

    #l2 = trial.suggest_float('l2', 0.007, 0.012)
    l2 = 0.01
    
    beta1 = trial.suggest_float('beta1', 0.9, 1)
    beta2 = trial.suggest_float('beta1', 0.9, 1)

    #learning_rate = trial.suggest_float("learning_rate", 1e-3, 0.5, log=True)
    learning_rate = 0.005
    batch_size = 256

    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
    #model.add(SpatialDropout1D(spat_dropout))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Bidirectional(CuDNNLSTM(lstm1_out, kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2))))
    #model.add(CuDNNLSTM(lstm2_out, kernel_regularizer=regularizers.L1L2(l1=l1, l2=l2)))
    model.add(Dense(6, activation='softmax'))
    # model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=learning_rate), metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate, beta_1=beta1, beta_2=beta2), metrics=['accuracy'])
    model.build(X_train.shape[1])
    #y_pred = model.predict(X_test).argmax(axis=1)

    callbacks = [
        #tf.keras.callbacks.EarlyStopping("val_accuracy", patience=3, min_delta=0.003),
        TFKerasPruningCallback(trial, "val_accuracy"),
    ]
    history = model.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=1, validation_data=(X_test, y_val),
                        callbacks=callbacks, )
    return history.history["val_accuracy"][-1]

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=2, n_min_trials=3, n_warmup_steps=3))
study.optimize(objective, n_trials=30)

df = study.trials_dataframe()
df.to_csv('data/study_two_layers_lr.csv')
study.best_params  

[I 2024-04-14 07:49:49,476] A new study created in memory with name: no-name-dcf76ddf-e96f-4670-aca0-e67bc3137adb


Epoch 1/20
35/35 [==============================] - 4s 57ms/step - loss: 8.5684 - accuracy: 0.4425 - val_loss: 5.2821 - val_accuracy: 0.4581
Epoch 2/20
35/35 [==============================] - 1s 42ms/step - loss: 4.6626 - accuracy: 0.5627 - val_loss: 4.3424 - val_accuracy: 0.5161
Epoch 3/20
35/35 [==============================] - 1s 42ms/step - loss: 3.8883 - accuracy: 0.6010 - val_loss: 3.6929 - val_accuracy: 0.5548
Epoch 4/20
35/35 [==============================] - 2s 45ms/step - loss: 3.3902 - accuracy: 0.6247 - val_loss: 3.3558 - val_accuracy: 0.5742
Epoch 5/20
35/35 [==============================] - 2s 46ms/step - loss: 3.0969 - accuracy: 0.6551 - val_loss: 3.3592 - val_accuracy: 0.5742
Epoch 6/20
35/35 [==============================] - 2s 46ms/step - loss: 2.9902 - accuracy: 0.6702 - val_loss: 3.1165 - val_accuracy: 0.5656
Epoch 7/20
35/35 [==============================] - 2s 44ms/step - loss: 2.8773 - accuracy: 0.6823 - val_loss: 3.1780 - val_accuracy: 0.5505
Epoch 8/20
35

[I 2024-04-14 07:50:22,739] Trial 0 finished with value: 0.5591397881507874 and parameters: {'beta1': 0.9443623585865634, 'learning_rate': 0.06799169380948811}. Best is trial 0 with value: 0.5591397881507874.


Epoch 1/20
35/35 [==============================] - 4s 60ms/step - loss: 11.4226 - accuracy: 0.3966 - val_loss: 7.1129 - val_accuracy: 0.3914
Epoch 2/20
35/35 [==============================] - 2s 44ms/step - loss: 6.5553 - accuracy: 0.4359 - val_loss: 4.5555 - val_accuracy: 0.5441
Epoch 3/20
35/35 [==============================] - 2s 43ms/step - loss: 4.2736 - accuracy: 0.5460 - val_loss: 4.1404 - val_accuracy: 0.5570
Epoch 4/20
35/35 [==============================] - 2s 45ms/step - loss: 3.3999 - accuracy: 0.5684 - val_loss: 3.0711 - val_accuracy: 0.5742
Epoch 5/20
35/35 [==============================] - 2s 45ms/step - loss: 2.8087 - accuracy: 0.5895 - val_loss: 2.4166 - val_accuracy: 0.5828
Epoch 6/20
35/35 [==============================] - 2s 44ms/step - loss: 2.2823 - accuracy: 0.6193 - val_loss: 2.1488 - val_accuracy: 0.5763
Epoch 7/20
35/35 [==============================] - 2s 45ms/step - loss: 1.9393 - accuracy: 0.6315 - val_loss: 1.8787 - val_accuracy: 0.5914
Epoch 8/20
3

[I 2024-04-14 07:50:56,212] Trial 1 finished with value: 0.602150559425354 and parameters: {'beta1': 0.9878168256338388, 'learning_rate': 0.0052763532332524956}. Best is trial 1 with value: 0.602150559425354.


...

Epoch 1/20
35/35 [==============================] - 3s 58ms/step - loss: 8.4132 - accuracy: 0.4524 - val_loss: 2.8695 - val_accuracy: 0.5441
Epoch 2/20
35/35 [==============================] - 2s 45ms/step - loss: 2.2028 - accuracy: 0.5612 - val_loss: 1.6217 - val_accuracy: 0.5720
Epoch 3/20
35/35 [==============================] - 2s 46ms/step - loss: 1.4533 - accuracy: 0.6054 - val_loss: 1.4143 - val_accuracy: 0.5613
Epoch 4/20
35/35 [==============================] - 2s 45ms/step - loss: 1.2967 - accuracy: 0.6362 - val_loss: 1.3586 - val_accuracy: 0.5699
Epoch 5/20
35/35 [==============================] - 2s 44ms/step - loss: 1.2252 - accuracy: 0.6545 - val_loss: 1.3742 - val_accuracy: 0.5892
Epoch 6/20
35/35 [==============================] - 2s 44ms/step - loss: 1.1628 - accuracy: 0.6788 - val_loss: 1.3311 - val_accuracy: 0.6000
Epoch 7/20
35/35 [==============================] - 2s 43ms/step - loss: 1.0980 - accuracy: 0.7080 - val_loss: 1.3745 - val_accuracy: 0.6043
Epoch 8/20
35

[I 2024-04-14 07:56:53,047] Trial 20 pruned. Trial was pruned at epoch 7.


Epoch 1/20
35/35 [==============================] - 4s 62ms/step - loss: 14.8565 - accuracy: 0.4365 - val_loss: 5.6452 - val_accuracy: 0.5161
Epoch 2/20
35/35 [==============================] - 2s 46ms/step - loss: 3.0588 - accuracy: 0.5162 - val_loss: 1.8313 - val_accuracy: 0.5548
Epoch 3/20
35/35 [==============================] - 2s 44ms/step - loss: 1.4956 - accuracy: 0.5737 - val_loss: 1.2711 - val_accuracy: 0.5699
Epoch 4/20
35/35 [==============================] - 2s 43ms/step - loss: 1.1998 - accuracy: 0.6079 - val_loss: 1.1672 - val_accuracy: 0.6022
Epoch 5/20
35/35 [==============================] - 2s 46ms/step - loss: 1.1161 - accuracy: 0.6349 - val_loss: 1.1823 - val_accuracy: 0.6086
Epoch 6/20
35/35 [==============================] - 2s 45ms/step - loss: 1.0820 - accuracy: 0.6522 - val_loss: 1.1621 - val_accuracy: 0.5935
Epoch 7/20
35/35 [==============================] - 2s 46ms/step - loss: 1.0317 - accuracy: 0.6704 - val_loss: 1.1721 - val_accuracy: 0.5978
Epoch 8/20
3

[I 2024-04-14 07:57:26,870] Trial 21 finished with value: 0.5935483574867249 and parameters: {'beta1': 0.9292815195760659, 'learning_rate': 0.0029114117571686403}. Best is trial 8 with value: 0.6236559152603149.


Epoch 1/20
35/35 [==============================] - 4s 60ms/step - loss: 19.7656 - accuracy: 0.3994 - val_loss: 12.8100 - val_accuracy: 0.5161
Epoch 2/20
35/35 [==============================] - 2s 44ms/step - loss: 8.3603 - accuracy: 0.4606 - val_loss: 4.5127 - val_accuracy: 0.5140
Epoch 3/20
35/35 [==============================] - 1s 42ms/step - loss: 2.7528 - accuracy: 0.5007 - val_loss: 1.8136 - val_accuracy: 0.5183
Epoch 4/20
35/35 [==============================] - 2s 44ms/step - loss: 1.5302 - accuracy: 0.5435 - val_loss: 1.3132 - val_accuracy: 0.5634
Epoch 5/20
 9/35 [======>.......................] - ETA: 1s - loss: 1.2555 - accuracy: 0.5751